# Sanofi - Accumulations

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("2019 Transparency Report CH.pdf", pages='all', lattice=True, multitable=True)

## Format Table

In [10]:
df_export = df.copy()

#Drop Columns
df_export = df_export.drop(df_export.columns[8:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#Shift
df_export[df_export['type'].isna()] = df_export[df_export['type'].isna()].shift(-1, axis='columns')

amount = df_export[df_export.type.str.contains('Aggregate amount ', na=False)]
recipients = df_export[df_export.type.str.contains('Number of Recipients', na=False)]
rnd = df_export[df_export.type.str.contains('Transfers of Value', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Cleanup Numbers and sum
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Fix Sum
df_export[df_export['type'].str.contains('Number of Recipients')] = np.NaN

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#Fix RnD
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

#Export
export_acumulations(df_export, 'sanofi')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')